## Load data and take the column that interested

## Description
In this assignment you must read in a file of metropolitan regions and associated sports teams from [assets/wikipedia_data.html](assets/wikipedia_data.html) and answer some questions about each metropolitan region. Each of these regions may have one or more teams from the "Big 4": NFL (football, in [assets/nfl.csv](assets/nfl.csv)), MLB (baseball, in [assets/mlb.csv](assets/mlb.csv)), NBA (basketball, in [assets/nba.csv](assets/nba.csv) or NHL (hockey, in [assets/nhl.csv](assets/nhl.csv)). Please keep in mind that all questions are from the perspective of the metropolitan region, and that this file is the "source of authority" for the location of a given sports team. Thus teams which are commonly known by a different area (e.g. "Oakland Raiders") need to be mapped into the metropolitan region given (e.g. San Francisco Bay Area). This will require some human data understanding outside of the data you've been given (e.g. you will have to hand-code some names, and might need to google to find out where teams are)!

For each sport I would like you to answer the question: **what is the win/loss ratio's correlation with the population of the city it is in?** Win/Loss ratio refers to the number of wins over the number of wins plus the number of losses. Remember that to calculate the correlation with [`pearsonr`](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.pearsonr.html), so you are going to send in two ordered lists of values, the populations from the wikipedia_data.html file and the win/loss ratio for a given sport in the same order. Average the win/loss ratios for those cities which have multiple teams of a single sport. Each sport is worth an equal amount in this assignment (20%\*4=80%) of the grade for this assignment. You should only use data **from year 2018** for your analysis -- this is important!


In [8]:
import numpy as np
import pandas as pd
from scipy import stats
import re

In [9]:
tabs = pd.read_html("./datasets/Wikipedia data.html")
df = tabs[1]
cities = df.iloc[:-1, [0, 3, 5, 6, 7, 8]]
cities.head(3)

,Metropolitan area,Population (2016 est.)[8],NFL,MLB,NBA,NHL
0,New York City,20153634,Giants Jets[note 1],Yankees Mets[note 2],Knicks Nets,Rangers Islanders Devils[note 3]
1,Los Angeles,13310447,Rams Chargers[note 4],Dodgers Angels,Lakers Clippers,Kings Ducks
2,San Francisco Bay Area,6657982,49ers Raiders[note 6],Giants Athletics,Warriors,Sharks[note 7]


### __Question 1__: Calculate the win/loss ratio's correlation with the population of the city it is in for the __NHL__ using __2018__ data.

In [10]:
nhl = pd.read_csv('./datasets/NHL data.csv')
nhl.head(3)

,team,GP,W,L,OL,PTS,PTS%,GF,GA,SRS,SOS,RPt%,ROW,year,League
0,Atlantic Division,Atlantic Division,Atlantic Division,Atlantic Division,Atlantic Division,Atlantic Division,Atlantic Division,Atlantic Division,Atlantic Division,Atlantic Division,Atlantic Division,Atlantic Division,Atlantic Division,2018,NHL
1,Tampa Bay Lightning*,82,54,23,5,113,.689,296,236,0.66,-0.07,.634,48,2018,NHL
2,Boston Bruins*,82,50,20,12,112,.683,270,214,0.62,-0.07,.610,47,2018,NHL


In [11]:
area_nhl = {'Tampa Bay Lightning':'Tampa Bay Area',
     'Boston Bruins':'Boston',
     'Toronto Maple Leafs':'Toronto',
     'Florida Panthers':'Miami–Fort Lauderdale',
     'Detroit Red Wings':'Detroit',
     'Montreal Canadiens':'Montreal',
     'Ottawa Senators':'Ottawa',
     'Buffalo Sabres':'Buffalo',
     'Washington Capitals':'Washington, D.C.',
     'Pittsburgh Penguins':'Pittsburgh',
     'Philadelphia Flyers':'Philadelphia',
     'Columbus Blue Jackets':'Columbus',
     'New Jersey Devils':'New York City',
     'Carolina Hurricanes':'Raleigh',
     'New York Islanders':'New York City',
     'New York Rangers':'New York City',
     'Nashville Predators':'Nashville',
     'Winnipeg Jets':'Winnipeg',
     'Minnesota Wild':'Minneapolis–Saint Paul',
     'Colorado Avalanche':'Denver',
     'St. Louis Blues':'St. Louis',
     'Dallas Stars':'Dallas–Fort Worth',
     'Chicago Blackhawks':'Chicago',
     'Vegas Golden Knights':'Las Vegas',
     'Anaheim Ducks':'Los Angeles',
     'San Jose Sharks':'San Francisco Bay Area',
     'Los Angeles Kings':'Los Angeles',
     'Calgary Flames':'Calgary',
     'Edmonton Oilers':'Edmonton',
     'Vancouver Canucks':'Vancouver',
     'Arizona Coyotes':'Phoenix'}

In [12]:
def remove_asterisk(text):
  return text.replace('*', '')

def remove_notes(text):
  return re.sub(r'\[note \d+\]', '', text)

In [14]:
cities = cities[['Metropolitan area','Population (2016 est.)[8]']]

,Metropolitan area,Population (2016 est.)[8]
0,New York City,20153634
1,Los Angeles,13310447
2,San Francisco Bay Area,6657982


In [15]:
nhl = nhl[['team','W', 'L', 'year']]
nhl.head(3)

,team,W,L,year
0,Atlantic Division,Atlantic Division,Atlantic Division,2018
1,Tampa Bay Lightning*,54,23,2018
2,Boston Bruins*,50,20,2018


In [25]:
# drop somw rows and filter 2018 year
nhl_2018 = nhl[nhl['year'] == 2018]
nhl_2018 = nhl_2018.drop([0, 9, 18, 26])
nhl_2018 = nhl_2018.reset_index(drop=True)

In [38]:
# remove asterisk in nhl data
nhl_2018['team'] = nhl_2018['team'].apply(remove_asterisk)
# convert the team according to metropolitan area
nhl_2018['team_area'] = nhl_2018['team']
nhl_2018['team_area'] = nhl_2018['team_area'].map(area_nhl)

,team,W,L,year,team_area
0,Tampa Bay Lightning,54,23,2018,Tampa Bay Area
1,Boston Bruins,50,20,2018,Boston
2,Toronto Maple Leafs,49,26,2018,Toronto


In [42]:
q1_df = pd.merge(cities, nhl_2018, left_on='Metropolitan area', right_on='team_area', how='inner')
# convert W, L, Population to numeric
q1_df['W'] = pd.to_numeric(q1_df['W'])
q1_df['L'] = pd.to_numeric(q1_df['L'])
q1_df['Population (2016 est.)[8]'] = pd.to_numeric(q1_df['Population (2016 est.)[8]'])
# calculate the W/L ratio
q1_df['W/L'] = q1_df['W'] / (q1_df['W'] + q1_df['L'])

In [45]:
q1_df['Metropolitan area'].unique().shape

(28,)

In [ ]:
# group the area and calculate the mean of them
q1_df = q1_df.groupby('Metropolitan area').agg({'Population (2016 est.)[8]': ['mean'], 'W/L': ['mean']})

In [65]:
q1_cor = stats.pearsonr(list(q1_df['Population (2016 est.)[8]']['mean']), list(q1_df['W/L']['mean']))

In [66]:
q1_cor

PearsonRResult(statistic=0.012486162921209881, pvalue=0.9497182859911808)

### __Question 2__: For this question, calculate the win/loss ratio's correlation with the population of the city it is in for the __NBA__ using __2018__ data.

In [68]:
nba = pd.read_csv('./datasets/NBA data.csv')
nba.head(3)

,team,W,L,W/L%,GB,PS/G,PA/G,SRS,year,League
0,Toronto Raptors* (1),59,23,0.72,—,111.7,103.9,7.29,2018,NBA
1,Boston Celtics* (2),55,27,0.6709999999999999,4.0,104.0,100.4,3.23,2018,NBA
2,Philadelphia 76ers* (3),52,30,0.634,7.0,109.8,105.3,4.3,2018,NBA


In [80]:
nba_area = {'Toronto Raptors':'Toronto',
     'Boston Celtics':'Boston',
     'Philadelphia 76ers':'Philadelphia',
     'Cleveland Cavaliers':'Cleveland',
     'Indiana Pacers':'Indianapolis',
     'Miami Heat':'Miami–Fort Lauderdale',
     'Milwaukee Bucks':'Milwaukee',
     'Washington Wizards':'Washington, D.C.',
     'Detroit Pistons':'Detroit',
     'Charlotte Hornets':'Charlotte',
     'New York Knicks':'New York City',
     'Brooklyn Nets':'New York City',
     'Chicago Bulls':'Chicago',
     'Orlando Magic':'Orlando',
     'Atlanta Hawks':'Atlanta',
     'Houston Rockets':'Houston',
     'Golden State Warriors':'San Francisco Bay Area',
     'Portland Trail Blazers':'Portland',
     'Oklahoma City Thunder':'Oklahoma City',
     'Utah Jazz':'Salt Lake City',
     'New Orleans Pelicans':'New Orleans',
     'San Antonio Spurs':'San Antonio',
     'Minnesota Timberwolves':'Minneapolis–Saint Paul',
     'Denver Nuggets':'Denver',
     'Los Angeles Clippers':'Los Angeles',
     'Los Angeles Lakers':'Los Angeles',
     'Sacramento Kings':'Sacramento',
     'Dallas Mavericks':'Dallas–Fort Worth',
     'Memphis Grizzlies':'Memphis',
     'Phoenix Suns':'Phoenix'} 

In [76]:
def clean_team_nba_name(text):
  pattern = r'\s*\*\s*|\s*\(\d+\)\s*'
  return re.sub(pattern, '', text).strip()

In [82]:
# clean the team_name
nba_2018 = nba[nba['year'] == 2018]
nba_2018.loc[:, 'team'] = nba_2018['team'].apply(clean_team_nba_name)
# take out the column that we are interested
nba_2018 = nba_2018[['team', 'W/L%']]

In [84]:
nba_2018['team_area'] = nba_2018['team']
nba_2018['team_area'] = nba_2018['team_area'].map(nba_area)
nba_2018

,team,W/L%,team_area
0,Toronto Raptors,0.72,Toronto
1,Boston Celtics,0.6709999999999999,Boston
2,Philadelphia 76ers,0.634,Philadelphia
3,Cleveland Cavaliers,0.61,Cleveland
4,Indiana Pacers,0.585,Indianapolis
5,Miami Heat,0.537,Miami–Fort Lauderdale
6,Milwaukee Bucks,0.537,Milwaukee
7,Washington Wizards,0.524,"Washington, D.C."
8,Detroit Pistons,0.47600000000000003,Detroit
9,Charlotte Hornets,0.439,Charlotte


In [94]:
q2_df = pd.merge(cities, nba_2018, left_on='Metropolitan area', right_on='team_area')
q2_df['W/L%'] = pd.to_numeric(q2_df['W/L%'])
q2_df['Population (2016 est.)[8]'] = pd.to_numeric(q2_df['Population (2016 est.)[8]'])
q2_df = q2_df.groupby('Metropolitan area').agg({'W/L%': ['mean'], 'Population (2016 est.)[8]': ['mean']})
q2_df

,W/L%,Population (2016 est.)[8]
,mean,mean
Metropolitan area,,
Atlanta,0.2930,5789700.0
Boston,0.6710,4794447.0
Charlotte,0.4390,2474314.0
Chicago,0.3290,9512999.0
Cleveland,0.6100,2055612.0
Dallas–Fort Worth,0.2930,7233323.0
Denver,0.5610,2853077.0
Detroit,0.4760,4297617.0


In [98]:
q2_cor = stats.pearsonr(list(q2_df['Population (2016 est.)[8]']['mean']), list(q2_df['W/L%']['mean']))

In [99]:
q2_cor

PearsonRResult(statistic=-0.17636350642182938, pvalue=0.36932106185547353)

## __Question 3__: For this question, calculate the win/loss ratio's correlation with the population of the city it is in for the MLB using 2018 data.

In [102]:
mlb_area = {'Boston Red Sox':'Boston',
     'New York Yankees':'New York City',
     'Tampa Bay Rays':'Tampa Bay Area',
     'Toronto Blue Jays':'Toronto',
     'Baltimore Orioles':'Baltimore',
     'Cleveland Indians':'Cleveland',
     'Minnesota Twins':'Minneapolis–Saint Paul',
     'Detroit Tigers':'Detroit',
     'Chicago White Sox':'Chicago',
     'Kansas City Royals':'Kansas City',
     'Houston Astros':'Houston',
     'Oakland Athletics':'San Francisco Bay Area',
     'Seattle Mariners':'Seattle',
     'Los Angeles Angels':'Los Angeles',
     'Texas Rangers':'Dallas–Fort Worth',
     'Atlanta Braves':'Atlanta',
     'Washington Nationals':'Washington, D.C.',
     'Philadelphia Phillies':'Philadelphia',
     'New York Mets':'New York City',
     'Miami Marlins':'Miami–Fort Lauderdale',
     'Milwaukee Brewers':'Milwaukee',
     'Chicago Cubs':'Chicago',
     'St. Louis Cardinals':'St. Louis',
     'Pittsburgh Pirates':'Pittsburgh',
     'Cincinnati Reds':'Cincinnati',
     'Los Angeles Dodgers':'Los Angeles',
     'Colorado Rockies':'Denver',
     'Arizona Diamondbacks':'Phoenix',
     'San Francisco Giants':'San Francisco Bay Area',
     'San Diego Padres':'San Diego'}
mlb = pd.read_csv('./datasets/MLB data.csv')
mlb.head()

,team,W,L,W-L%,GB,year,League
0,Boston Red Sox,108,54,0.667,--,2018,MLB
1,New York Yankees,100,62,0.617,8.0,2018,MLB
2,Tampa Bay Rays,90,72,0.556,18.0,2018,MLB
3,Toronto Blue Jays,73,89,0.451,35.0,2018,MLB
4,Baltimore Orioles,47,115,0.290,61.0,2018,MLB


In [111]:
mlb_2018 = mlb[mlb['year'] == 2018]
mlb_2018 = mlb_2018[['team', 'W-L%']]
mlb_2018['team_area'] = mlb_2018['team']
mlb_2018['team_area'] = mlb_2018['team_area'].map(mlb_area)

In [ ]:
q3_df = pd.merge(cities, mlb_2018, left_on='Metropolitan area', right_on='team_area', how='inner' )
q3_df['Population (2016 est.)[8]'] =  pd.to_numeric(q3_df['Population (2016 est.)[8]'])
q3_df['W-L%'] = pd.to_numeric(q3_df['W-L%'])
q3_df = q3_df.groupby('Metropolitan area').agg({'Population (2016 est.)[8]': ['mean'], 'W-L%': ['mean']})
q3_df


In [114]:
q3_cor = stats.pearsonr(list(q3_df['Population (2016 est.)[8]']['mean']), list(q3_df['W-L%']['mean']))
q3_cor

PearsonRResult(statistic=0.150037374754095, pvalue=0.46442827201123316)

### __Question 4__: For this question, calculate the win/loss ratio's correlation with the population of the city it is in for the __NFL__ using __2018__ data.

In [122]:
nfl_area = {'New England Patriots':'Boston',
     'Miami Dolphins':'Miami–Fort Lauderdale',
     'Buffalo Bills':'Buffalo',
     'New York Jets':'New York City',
     'Baltimore Ravens':'Baltimore',
     'Pittsburgh Steelers':'Pittsburgh',
     'Cleveland Browns':'Cleveland',
     'Cincinnati Bengals':'Cincinnati',
     'Houston Texans':'Houston',
     'Indianapolis Colts':'Indianapolis',
     'Tennessee Titans':'Nashville',
     'Jacksonville Jaguars':'Jacksonville',
     'Kansas City Chiefs':'Kansas City',
     'Los Angeles Chargers':'Los Angeles',
     'Denver Broncos':'Denver',
     'Oakland Raiders':'San Francisco Bay Area',
     'Dallas Cowboys':'Dallas–Fort Worth',
     'Philadelphia Eagles':'Philadelphia',
     'Washington Redskins':'Washington, D.C.',
     'New York Giants':'New York City',
     'Chicago Bears':'Chicago',
     'Minnesota Vikings':'Minneapolis–Saint Paul',
     'Green Bay Packers':'Green Bay',
     'Detroit Lions':'Detroit',
     'New Orleans Saints':'New Orleans',
     'Carolina Panthers':'Charlotte',
     'Atlanta Falcons':'Atlanta',
     'Tampa Bay Buccaneers':'Tampa Bay Area',
     'Los Angeles Rams':'Los Angeles',
     'Seattle Seahawks':'Seattle',
     'San Francisco 49ers':'San Francisco Bay Area',
     'Arizona Cardinals':'Phoenix'}
nfl = pd.read_csv('./datasets/NFL data.csv')
nfl.head()

,DSRS,L,League,MoV,OSRS,PA,PD,PF,SRS,SoS,T,W,W-L%,team,year
0,AFC East,AFC East,NFL,AFC East,AFC East,AFC East,AFC East,AFC East,AFC East,AFC East,AFC East,AFC East,AFC East,AFC East,2018
1,2.1,5,NFL,6.9,3.1,325,111,436,5.2,-1.8,0,11,.688,New England Patriots*,2018
2,-5.2,9,NFL,-7.1,-3.6,433,-114,319,-8.8,-1.7,0,7,.438,Miami Dolphins,2018
3,-0.6,10,NFL,-6.6,-6.3,374,-105,269,-6.9,-0.3,0,6,.375,Buffalo Bills,2018
4,-5.9,12,NFL,-6.8,-2.0,441,-108,333,-7.8,-1.1,0,4,.250,New York Jets,2018


In [124]:
nfl_2018 = nfl[nfl['year'] == 2018]
nfl_2018  = nfl_2018.drop(list(range(0, len(nfl_2018), 5))).reset_index(drop=True)

In [133]:
pattern = r'\s*[\*\+]\s*'
def remove_asterisk_add_symbol(text):
  return re.sub(pattern, '', text).strip()
nfl_2018.loc[:,'team'] = nfl_2018['team'].apply(remove_asterisk_add_symbol)
nfl_2018 = nfl_2018[['team', 'W-L%']]
nfl_2018['W-L%'] = pd.to_numeric(nfl_2018['W-L%'])

In [136]:
nfl_2018['team_area'] = nfl_2018['team']
nfl_2018['team_area'] = nfl_2018['team_area'].map(nfl_area)

In [141]:
q4_df = pd.merge(cities, nfl_2018, right_on='team_area', left_on='Metropolitan area', how='inner')
q4_df['Population (2016 est.)[8]'] = pd.to_numeric(q4_df['Population (2016 est.)[8]'])
q4_df = q4_df[['Population (2016 est.)[8]', 'W-L%', 'Metropolitan area']]
q4_df = q4_df.groupby('Metropolitan area').agg({'Population (2016 est.)[8]': ['mean'], 'W-L%': ['mean']})
q4_df

,Population (2016 est.)[8],W-L%
,mean,mean
Metropolitan area,,
Atlanta,5789700.0,0.4380
Baltimore,2798886.0,0.6250
Boston,4794447.0,0.6880
Buffalo,1132804.0,0.3750
Charlotte,2474314.0,0.4380
Chicago,9512999.0,0.7500
Cincinnati,2165139.0,0.3750
Cleveland,2055612.0,0.4690


In [145]:
q4_cor = stats.pearsonr(list(q4_df['Population (2016 est.)[8]']['mean']), list(q4_df['W-L%']['mean']))
q4_cor

PearsonRResult(statistic=0.00428214143639298, pvalue=0.9824114740736539)

## Question 5
In this question I would like you to explore the hypothesis that **given that an area has two sports teams in different sports, those teams will perform the same within their respective sports**. How I would like to see this explored is with a series of paired t-tests (so use [`ttest_rel`](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.ttest_rel.html)) between all pairs of sports. Are there any sports where we can reject the null hypothesis? Again, average values where a sport has multiple teams in one region. Remember, you will only be including, for each sport, cities which have teams engaged in that sport, drop others as appropriate. This question is worth 20% of the grade for this assignment.

- NHL: q1_df
- NBA: q2_df
- MLB: q3_df
- NFL: q4_df


In [174]:
nhl = q1_df['W/L']
nhl.columns = ['NHL']
nba= q2_df['W/L%']
nba.columns = ['NBA']
mlb = q3_df['W-L%']
mlb.columns = ['MLB']
nfl = q4_df['W-L%']
nfl.columns = ['NFL']

In [183]:
# t-test between nba and nfl
nba_nfl= pd.merge(nba, nfl, on='Metropolitan area', how='inner')
pval_nba_nfl = stats.ttest_rel(nba_nfl['NBA'], nba_nfl['NFL'])[1]

# t-test between nba and nhl
nba_nhl = pd.merge(nba, nhl, on='Metropolitan area', how='inner')
pval_nba_nhl = stats.ttest_rel(nba_nhl['NBA'], nba_nhl['NHL'])[1]

# t-test between nba and mlb
nba_mlb = pd.merge(nba, mlb, on='Metropolitan area', how='inner')
pval_nba_mlb = stats.ttest_rel(nba_mlb['NBA'], nba_mlb['MLB'] )[1]

# t-test between nfl and nhl
nfl_nhl = pd.merge(nfl, nhl, on='Metropolitan area', how='inner')
pval_nfl_nhl = stats.ttest_rel(nfl_nhl['NFL'], nfl_nhl['NHL'])[1]

# t-test between nfl and mlb
nfl_mlb = pd.merge(nfl, mlb, on='Metropolitan area', how='inner')
pval_nfl_mlb = stats.ttest_rel(nfl_mlb['NFL'], nfl_mlb['MLB'])[1]

# t-test between nhl and mlb
nhl_mlb = pd.merge(nhl, mlb, on='Metropolitan area', how='inner')
pval_nhl_mlb = stats.ttest_rel(nhl_mlb['NHL'], nhl_mlb['MLB'])[1]

In [184]:
pv = {
  'NFL': {
    'NFL': np.nan,
    'NBA': pval_nba_nfl,
    'NHL': pval_nfl_nhl,
    'MLB': pval_nfl_mlb,
  },
  'NBA': {
    'NFL': pval_nba_nfl,
    'NBA': np.nan,
    'NHL': pval_nba_nhl,
    'MLB': pval_nba_mlb,
  },
  'NHL': {
    'NFL': pval_nfl_nhl,
    'NBA': pval_nba_nhl,
    'NHL': np.nan,
    'MLB': pval_nhl_mlb,
  },
  'MLB': {
    'NFL': pval_nfl_mlb,
    'NBA': pval_nba_mlb,
    'NHL': pval_nhl_mlb,
    'MLB': np.nan
  }
}
q_5_t_test = pd.DataFrame(pv)
q_5_t_test

,NFL,NBA,NHL,MLB
NFL,NaN,0.937509,0.030318,0.803459
NBA,0.937509,NaN,0.022386,0.949566
NHL,0.030318,0.022386,NaN,0.000703
MLB,0.803459,0.949566,0.000703,NaN
